In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange

device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.manual_seed(1984)


from init import get_AEmodel_cfg
from core.models import get_model
category = "mugs"
# category = "kit4cates"
# category = "chairs"
config_Only_model = get_AEmodel_cfg()
ModelClass = get_model(config_Only_model["model"]["model_name"])
model = ModelClass(config_Only_model)
ckpt_path = f"/home/ziran/se3/EFEM/weights/{category}.pt"
ckpt = torch.load(ckpt_path)
model.network.load_state_dict(ckpt['model_state_dict'])
model.network = model.network.to(device)

bs = 1


Using device: cuda:4


In [ ]:
import pickle
with open(f'./dev_utils/test.sdf', 'rb') as f:
    result = pickle.load(f)
# 分离点云位置和SDF值
points = result['sdf'][:, :3]  # 取前三列为点云位置
sdf_values = result['sdf'][:, 3]  # 取第四列为SDF值
normals = result['normals']
# 检查结果
print("Points shape:", points.shape)
print("SDF Values shape:", sdf_values.shape)
print("Normals shape:", normals.shape)

query = torch.from_numpy(points).unsqueeze(0).repeat(bs, 1, 1).to(device=device, dtype=torch.float32)
sdf_values = torch.from_numpy(sdf_values).unsqueeze(0).repeat(bs, 1).to(device=device, dtype=torch.float32)
query_mask = torch.ones(query.shape[:2]).bool()
normals = torch.from_numpy(normals).unsqueeze(0).repeat(bs, 1, 1).to(device=device, dtype=torch.float32)
print("query max,min: ",query.max(), query.min())
print("query_sdf max,min: ",sdf_values.max(), sdf_values.min())
query.shape, query_mask.shape, sdf_values.shape

In [ ]:
def get_recon_score(decoder_input, sdf_gt):
    sdf_hat = model.network.decode(
        None,
        None,
        None,
        return_sdf=True,
        decoder_input = decoder_input
    )

    total_loss = torch.mean(torch.abs(sdf_hat - sdf_gt))
    return total_loss